In [ ]:
%pip install arviz
%pip install numpyro
%pip install seaborn

In [ ]:
import math
import os
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

from matplotlib import gridspec
from matplotlib.patches import Rectangle
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator, FixedLocator
from matplotlib import rc

import jax
import jax.numpy as jnp
from jax import nn, random, vmap
from jax.scipy.special import expit

import numpyro as npr
import numpyro.distributions as dist
import numpyro.optim as optim
from numpyro.diagnostics import print_summary
from numpyro.infer import MCMC, NUTS, Predictive, SVI, Trace_ELBO, log_likelihood
from numpyro.infer.autoguide import AutoLaplaceApproximation
from numpyro import sample, deterministic
from numpyro.distributions.transforms import OrderedTransform

if "SVG" in os.environ:
    %config InlineBackend.figure_formats = ["svg"]
warnings.formatwarning = lambda message, category, *args, **kwargs: "{}: {}\n".format(
    category.__name__, message
)

npr.set_platform("cpu")
npr.set_host_device_count(10)
npr.enable_x64()

print('Running on Numpyro v{}'.format(npr.__version__))
print('Running on Seaborn v{}'.format(sns.__version__))

from IPython.display import Image

# color = '#87CEEB'
# initialize_plt_font()


RANDOM_SEED = 9876

In [ ]:
# color definitions
base_color = "#1696D2"
accent_color = "#FCB918"
neutral_gray= "#C6C6C6"
green_color = '#186b2b'
yellow_color = '#dea228'

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-white")
az.rcParams["stats.hdi_prob"] = 0.89
az.rcParams["stats.ic_scale"] = "deviance"
az.rcParams["stats.information_criterion"] = "loo"


pd.set_option('display.max_rows', None)

In [ ]:


# If it is run on Colab, put the exp1_data folder in the root of your Google drive and use this data path
# DATA_PATH = "drive/My Drive/QV_data"
# OUTPUT_PATH = "drive/My Drive/QV_data/Output/Intensity"
# MODEL_PATH = "drive/My Drive/QV_data/Model"
# If it is run on a local machine, use the following data path
DATA_PATH = ""
MODEL_PATH = ""
OUTPUT_PATH = ""

In [ ]:
from tqdm.autonotebook import tqdm

## Import Data

### Likert Data

In [ ]:
likert_raw = pd.read_csv(os.path.join(DATA_PATH, 'exp1_data', 'ALLOCATE_LIKERT_P1.csv')).sort_values(by=['userId'])
print(likert_raw.shape)
likert_raw.head()

In [ ]:
likert_d_raw = pd.read_csv(os.path.join(DATA_PATH, 'exp1_data', 'DONATION_LIKERT.csv')).sort_values(by=['userId'])
likert_d_raw.head()

### QV Data

In [ ]:
qv36_raw = pd.read_csv(os.path.join(DATA_PATH, 'exp1_data', 'QV_P1_36.csv')).sort_values(by=['userId'])
print(qv36_raw.shape)
qv36_raw.head()

In [ ]:
qv36_d_raw = pd.read_csv(os.path.join(DATA_PATH, 'exp1_data', 'DONATION_QV36.csv')).sort_values(by=['userId'])
print(qv36_d_raw.shape)
qv36_d_raw.head()

In [ ]:
qv108_raw = pd.read_csv(os.path.join(DATA_PATH, 'exp1_data', 'QV_P1_108.csv')).sort_values(by=['userId'])
qv108_raw.head()

In [ ]:
qv108_d_raw = pd.read_csv(os.path.join(DATA_PATH, 'exp1_data', 'DONATION_QV108.csv')).sort_values(by=['userId'])
qv108_d_raw.head()

In [ ]:
qv324_raw = pd.read_csv(os.path.join(DATA_PATH, 'exp1_data', 'QV_P1_324.csv')).sort_values(by=['userId'])
qv324_raw.head()

In [ ]:
qv324_d_raw = pd.read_csv(os.path.join(DATA_PATH, 'exp1_data', 'DONATION_QV324.csv')).sort_values(by=['userId'])
print(qv324_d_raw.shape)
qv324_d_raw.head()

### Linear QV Data

In [ ]:
lqv36_raw = pd.read_csv(os.path.join(DATA_PATH, 'linearQV_rerun_data', 'LinearQV_36.csv')).sort_values(by=['userId'])
print(lqv36_raw.shape)
lqv36_raw.head()

In [ ]:
lqv36_d_raw = pd.read_csv(os.path.join(DATA_PATH, 'linearQV_rerun_data', 'LinearQV_36_Donation.csv')).sort_values(by=['userId'])
print(lqv36_d_raw.shape)
lqv36_d_raw.head()

In [ ]:
lqv108_raw = pd.read_csv(os.path.join(DATA_PATH, 'linearQV_rerun_data', 'LinearQV_108.csv')).sort_values(by=['userId'])
print(lqv108_raw.shape)
lqv108_raw.head()

In [ ]:
lqv108_d_raw = pd.read_csv(os.path.join(DATA_PATH, 'linearQV_rerun_data', 'LinearQV_108_Donation.csv')).sort_values(by=['userId'])
print(lqv108_d_raw.shape)
lqv108_d_raw.head()

In [ ]:
lqv324_raw = pd.read_csv(os.path.join(DATA_PATH, 'linearQV_rerun_data', 'LinearQV_324.csv')).sort_values(by=['userId'])
print(lqv324_raw.shape)
lqv324_raw.head()

In [ ]:
lqv324_d_raw = pd.read_csv(os.path.join(DATA_PATH, 'linearQV_rerun_data', 'LinearQV_324_Donation.csv')).sort_values(by=['userId'])
print(lqv324_d_raw.shape)
lqv324_d_raw.head()

### Unlimited QV

In [ ]:
unlimited_raw = pd.read_csv(os.path.join(DATA_PATH, 'unlimitedQV_data', 'UnlimitedQV.csv')).sort_values(by=['userId'])
print(unlimited_raw.shape)
unlimited_raw.head()

In [ ]:
vote_columns = ['pets','art','education','environment','health','human','international','faith','veteran']
def calculate_total_votes_credit_used(row, vote_columns):
  total_vote = 0
  total_credit = 0
  for col in vote_columns:
    total_vote += abs(row[col])
    total_credit += row[col] ** 2

  return total_vote, total_credit

unlimited_raw[['total_vote_spent', 'total_credit_spent']] = unlimited_raw.apply(calculate_total_votes_credit_used, axis=1, args=(vote_columns,)).apply(pd.Series)
unlimited_raw.head()

In [ ]:
unlimited_d_raw = pd.read_csv(os.path.join(DATA_PATH, 'unlimitedQV_data', 'UnlimitedQV_Donation.csv')).sort_values(by=['userId'])
print(unlimited_d_raw.shape)
unlimited_d_raw.head()

### Data Processing

In [ ]:
donate_columns = ['donation_a_pets','donation_b_art','donation_c_education','donation_d_environment','donation_e_health','donation_f_human','donation_g_international','donation_h_faith','donation_i_veteran']

# likert = likert_raw[vote_columns].to_numpy()-3 # Likert survey, normalized to -2 to 2
likert_d = likert_d_raw[donate_columns].to_numpy() # Likert donation

# qv36 = qv36_raw[vote_columns].to_numpy() # QV36 survey
qv36_d = qv36_d_raw[donate_columns].to_numpy() # QV36 donation

# qv108 = qv108_raw[vote_columns].to_numpy() # QV108 survey
qv108_d = qv108_d_raw[donate_columns].to_numpy() # QV108 donation

# qv324 = qv324_raw[vote_columns].to_numpy() # QV324 survey
qv324_d = qv324_d_raw[donate_columns].to_numpy() # QV324 donation

lqv36_d = lqv36_d_raw[donate_columns].to_numpy()
lqv108_d = lqv108_d_raw[donate_columns].to_numpy()
lqv324_d = lqv324_d_raw[donate_columns].to_numpy()

unlimited_d = unlimited_d_raw[donate_columns].to_numpy()

# print(likert[:5,:])

In [ ]:
# Function to remove participants with zero total donation
def filter_nonzero(survey_data_wid, donation_data_wid, donation_data):
    d_total = np.sum(donation_data, axis=1)
    d_bool = (d_total != 0)
    survey_f = survey_data_wid[d_bool]
    donation_f = donation_data_wid[d_bool]
    total_spent = d_total[d_bool]
    donation_f['total_spent'] = total_spent
    print(donation_f.head())
    return survey_f, donation_f

In [ ]:
# Remove participants with zero total donation and normalize donation amount
likert_f, likert_d_f = filter_nonzero(likert_raw, likert_d_raw, likert_d)
# likert_d_f_norm = normalize_donation_to_proportion(likert_d_f)

qv36_f, qv36_d_f = filter_nonzero(qv36_raw, qv36_d_raw, qv36_d)
print(qv36_f.shape)
# qv36_d_f_norm = normalize_donation_to_proportion(qv36_d_f)

qv108_f, qv108_d_f = filter_nonzero(qv108_raw, qv108_d_raw, qv108_d)
print(qv108_f.shape)
# qv108_d_f_norm = normalize_donation_to_proportion(qv108_d_f)

qv324_f, qv324_d_f = filter_nonzero(qv324_raw, qv324_d_raw, qv324_d)
print(qv324_f.shape)
# qv324_d_f_norm = normalize_donation_to_proportion(qv324_d_f)

lqv36_f, lqv36_d_f = filter_nonzero(lqv36_raw, lqv36_d_raw, lqv36_d)
print(lqv36_f.shape)
# qv36_d_f_norm = normalize_donation_to_proportion(qv36_d_f)

lqv108_f, lqv108_d_f = filter_nonzero(lqv108_raw, lqv108_d_raw, lqv108_d)
print(lqv108_f.shape)
# qv108_d_f_norm = normalize_donation_to_proportion(qv108_d_f)

lqv324_f, lqv324_d_f = filter_nonzero(lqv324_raw, lqv324_d_raw, lqv324_d)
print(lqv324_f.shape)
# qv324_d_f_norm = normalize_donation_to_proportion(qv324_d_f)

unlimited_f, unlimited_d_f = filter_nonzero(unlimited_raw, unlimited_d_raw, unlimited_d)
print(unlimited_f.shape)

In [ ]:
# function that calculates pair response differences
def survey_diff(qv_id, qv_survey_responses, condition, topic_columns):
    l = qv_id.shape[0]
    condition = pd.DataFrame([condition] * l, columns=['condition'])
    print(l)

    qv_pair_list = []
    for i in range(len(topic_columns)-1):
        topic_1 = topic_columns[i]
        topic_1_df = pd.DataFrame([topic_1]*l, columns=['topic_1'])

        for j in range(i+1, len(topic_columns)):
            topic_2 = topic_columns[j]
            topic_2_df = pd.DataFrame([topic_2]*l, columns=['topic_2'])

            topic_1_val = pd.DataFrame(qv_survey_responses[:,i], columns=['topic_1_val'])
            topic_2_val = pd.DataFrame(qv_survey_responses[:,j], columns=['topic_2_val'])

            vote_diff = qv_survey_responses[:,i] - qv_survey_responses[:,j]
            vote_diff_df = pd.DataFrame(vote_diff, columns=['vote_diff'])

            vote_diff_complete_df = pd.concat([qv_id.reset_index(), condition, topic_1_df, topic_1_val, topic_2_df, topic_2_val, vote_diff_df], axis=1)
            qv_pair_list.append(vote_diff_complete_df)

    # print(len(qv_pair_list))
    qv_pair = pd.concat(qv_pair_list, axis=0)
    print(qv_pair.shape)
    print(qv_pair.head())

    return qv_pair

In [ ]:
def normalize_diff(diffs, min_diff, max_diff):
  diffs_norm = 2 * (diffs - min_diff) / (max_diff - min_diff) - 1
  print(diffs_norm.min(), diffs_norm.max())
  return diffs_norm

In [ ]:
def normalize_diff_per_row(row, diff_col_name, total_col_name):
  diff = row[diff_col_name]
  total_used = row[total_col_name]
  if total_used != 0:
    diff_norm = 2 * (diff + total_used) / (total_used * 2) - 1
    if diff_norm < -1 or diff_norm > 1:
      print("out of range:")
      print(row)
  else:
    diff_norm = 0
  return diff_norm

#### QV

In [ ]:
# function that calculates pair credit differences
def credit_diff(qv_df, new_condition):
  columns = list(qv_df.columns)
  columns.remove("vote_diff")
  qv_credit_df = qv_df[columns]
  qv_credit_df['credit_diff'] = qv_credit_df['topic_1_val'] ** 2 - qv_credit_df['topic_2_val'] ** 2
  qv_credit_df['condition'] = new_condition

  return qv_credit_df

In [ ]:
qv36_pair = survey_diff(qv36_f[['userId', 'order']], qv36_f[vote_columns].to_numpy(), 'qv36', vote_columns)
qv36_pair_credit = credit_diff(qv36_pair, 'qv36_c')
# qv36_pair.to_csv(os.path.join(OUTPUT_PATH, 'qv36_pair.csv'))

qv36_pair['vote_diff_norm'] = normalize_diff(qv36_pair['vote_diff'], -8, 8) # +5 vs -3
qv36_pair['vote_diff_ordinal'] = qv36_pair['vote_diff'] + 8
qv36_pair_credit['credit_diff_norm'] = normalize_diff(qv36_pair_credit['credit_diff'], -36, 36)
print(qv36_pair.head())

In [ ]:
qv108_pair = survey_diff(qv108_f[['userId', 'order']], qv108_f[vote_columns].to_numpy(), 'qv108', vote_columns)
qv108_pair_credit = credit_diff(qv108_pair, 'qv108_c')
# qv108_pair.to_csv(os.path.join(OUTPUT_PATH, 'qv108_pair.csv'))

qv108_pair['vote_diff_norm'] = normalize_diff(qv108_pair['vote_diff'], -14, 14)
qv108_pair['vote_diff_ordinal'] = qv108_pair['vote_diff'] + 14
qv108_pair_credit['credit_diff_norm'] = normalize_diff(qv108_pair_credit['credit_diff'], -108, 108)
print(qv108_pair.head())

In [ ]:
qv324_pair = survey_diff(qv324_f[['userId', 'order']], qv324_f[vote_columns].to_numpy(), 'qv324', vote_columns)
qv324_pair_credit = credit_diff(qv324_pair, 'qv324_c')
# qv324_pair.to_csv(os.path.join(OUTPUT_PATH, 'qv324_pair.csv'))

qv324_pair['vote_diff_norm'] = normalize_diff(qv324_pair['vote_diff'], -25, 25)
qv324_pair['vote_diff_ordinal'] = qv324_pair['vote_diff'] + 25
qv324_pair_credit['credit_diff_norm'] = normalize_diff(qv324_pair_credit['credit_diff'], -324, 324)
print(qv324_pair.head())

In [ ]:
qv_pair = pd.concat([qv36_pair, qv108_pair, qv324_pair], axis=0).reset_index()[['userId', 'condition', 'order', 'topic_1', 'topic_1_val', 'topic_2', 'topic_2_val', 'vote_diff', 'vote_diff_norm', 'vote_diff_ordinal']]
print(qv_pair.head())

qv_c_pair = pd.concat([qv36_pair_credit, qv108_pair_credit, qv324_pair_credit], axis=0).reset_index()[['userId', 'condition', 'order', 'topic_1', 'topic_1_val', 'topic_2', 'topic_2_val', 'credit_diff', 'credit_diff_norm']]
qv_c_pair_renamed = qv_c_pair.rename(columns={"credit_diff": "vote_diff", "credit_diff_norm": "vote_diff_norm"})
print(qv_c_pair_renamed.head())

#### Likert

In [ ]:
likert_pair = survey_diff(likert_f[['userId', 'order']], likert_f[vote_columns].to_numpy(), 'likert', vote_columns)
# likert_pair.to_csv(os.path.join(OUTPUT_PATH, 'likert_pair.csv'))
likert_pair['vote_diff_norm'] = normalize_diff(likert_pair['vote_diff'], -4, 4)
likert_pair['vote_diff_ordinal'] = likert_pair['vote_diff'] + 4
print(likert_pair.head())

#### LQV

In [ ]:
lqv36_pair = survey_diff(lqv36_f[['userId', 'order']], lqv36_f[vote_columns].to_numpy(), 'lqv36', vote_columns)
# lqv36_pair.to_csv(os.path.join(OUTPUT_PATH, 'lqv36_pair.csv'))
lqv36_pair['vote_diff_norm'] = normalize_diff(lqv36_pair['vote_diff'], -36, 36)

In [ ]:
lqv108_pair = survey_diff(lqv108_f[['userId', 'order']], lqv108_f[vote_columns].to_numpy(), 'lqv108', vote_columns)
# lqv108_pair.to_csv(os.path.join(OUTPUT_PATH, 'lqv108_pair.csv'))
lqv108_pair['vote_diff_norm'] = normalize_diff(lqv108_pair['vote_diff'], -108, 108)

In [ ]:
lqv324_pair = survey_diff(lqv324_f[['userId', 'order']], lqv324_f[vote_columns].to_numpy(), 'lqv324', vote_columns)
# lqv324_pair.to_csv(os.path.join(OUTPUT_PATH, 'lqv324_pair.csv'))
lqv324_pair['vote_diff_norm'] = normalize_diff(lqv324_pair['vote_diff'], -324, 324)

#### Unlimited

In [ ]:
unlimited_pair = survey_diff(unlimited_f[['userId', 'order', 'total_vote_spent', 'total_credit_spent']], unlimited_f[vote_columns].to_numpy(), 'unlimited', vote_columns)
unlimited_pair_credit = credit_diff(unlimited_pair, 'unlimited_c')
print(unlimited_pair_credit.head())
# unlimited_pair.to_csv(os.path.join(OUTPUT_PATH, 'unlimited_pair.csv'))

# Votes for unlimited QV
unlimited_pair['vote_diff_norm'] = unlimited_pair.apply(normalize_diff_per_row, axis=1, args=('vote_diff', 'total_vote_spent',))
unlimited_pair.head()
print(unlimited_pair['vote_diff_norm'].min(), unlimited_pair['vote_diff_norm'].max())

# Credits for unlimited QV, column "credit_diff_norm" is renamed as "vote_diff_norm" to enable concatenation
unlimited_pair_credit['vote_diff_norm'] = unlimited_pair_credit.apply(normalize_diff_per_row, axis=1, args=('credit_diff', 'total_credit_spent',))
unlimited_pair_credit.head()
print(unlimited_pair_credit['vote_diff_norm'].min(), unlimited_pair['vote_diff_norm'].max())

#### Concatenate

In [ ]:
shared_cols = ['userId', 'condition', 'order', 'topic_1', 'topic_1_val', 'topic_2', 'topic_2_val', 'vote_diff', 'vote_diff_norm']
# unlimited_pair_selected = unlimited_pair[shared_cols]

all_pair = pd.concat([qv_pair, qv_c_pair_renamed, likert_pair, lqv36_pair, lqv108_pair, lqv324_pair, unlimited_pair, unlimited_pair_credit], axis=0).reset_index()[['userId', 'condition', 'order', 'topic_1', 'topic_1_val', 'topic_2', 'topic_2_val', 'vote_diff', 'vote_diff_norm', 'vote_diff_ordinal']]
all_pair['vote_diff_ordinal'] = all_pair['vote_diff_ordinal'].astype('Int64')

print(all_pair.head())
print(all_pair.shape)
all_pair.to_csv(os.path.join(OUTPUT_PATH, 'all_pair.csv'))

In [ ]:
frequency_df = all_pair.groupby(['userId', 'condition', 'topic_1', 'topic_2', 'vote_diff']).size().reset_index(name='Frequency')
duplicated_bool = frequency_df['Frequency'] > 1
rows_duplicated = frequency_df[duplicated_bool]
print(rows_duplicated.shape)

In [ ]:
all_pair_unique = all_pair.drop_duplicates(subset=['userId', 'condition', 'topic_1', 'topic_2', 'order', 'topic_1_val', 'topic_2_val', 'vote_diff'])
print(all_pair_unique.shape)

In [ ]:
ordinal_pair = all_pair_unique[all_pair_unique['condition'].isin(['qv36', 'qv108', 'qv324', 'likert'])]
ordinal_pair['vote_diff_ordinal'].value_counts()

#### Donation

In [ ]:
qv36_d_pair = survey_diff(qv36_d_f[['userId', 'total_spent']], qv36_d_f[donate_columns].to_numpy(), 'qv36', vote_columns)
# qv36_d_pair.to_csv(os.path.join(OUTPUT_PATH, 'qv36_d_pair.csv'))'
qv36_d_pair_credit = survey_diff(qv36_d_f[['userId', 'total_spent']], qv36_d_f[donate_columns].to_numpy(), 'qv36_c', vote_columns)

In [ ]:
qv108_d_pair = survey_diff(qv108_d_f[['userId', 'total_spent']], qv108_d_f[donate_columns].to_numpy(), 'qv108', vote_columns)
# qv108_d_pair.to_csv(os.path.join(OUTPUT_PATH, 'qv108_d_pair.csv'))
qv108_d_pair_credit = survey_diff(qv108_d_f[['userId', 'total_spent']], qv108_d_f[donate_columns].to_numpy(), 'qv108_c', vote_columns)

In [ ]:
qv324_d_pair = survey_diff(qv324_d_f[['userId', 'total_spent']], qv324_d_f[donate_columns].to_numpy(), 'qv324', vote_columns)
# qv324_d_pair.to_csv(os.path.join(OUTPUT_PATH, 'qv324_d_pair.csv'))
qv324_d_pair_credit = survey_diff(qv324_d_f[['userId', 'total_spent']], qv324_d_f[donate_columns].to_numpy(), 'qv324_c', vote_columns)

In [ ]:
likert_d_pair = survey_diff(likert_d_f[['userId', 'total_spent']], likert_d_f[donate_columns].to_numpy(), 'likert', vote_columns)
# likert_d_pair.to_csv(os.path.join(OUTPUT_PATH, 'likert_d_pair.csv'))

In [ ]:
lqv36_d_pair = survey_diff(lqv36_d_f[['userId', 'total_spent']], lqv36_d_f[donate_columns].to_numpy(), 'lqv36', vote_columns)
# lqv36_d_pair.to_csv(os.path.join(OUTPUT_PATH, 'lqv36_d_pair.csv'))

In [ ]:
lqv108_d_pair = survey_diff(lqv108_d_f[['userId', 'total_spent']], lqv108_d_f[donate_columns].to_numpy(), 'lqv108', vote_columns)
# lqv108_d_pair.to_csv(os.path.join(OUTPUT_PATH, 'lqv108_d_pair.csv'))

In [ ]:
lqv324_d_pair = survey_diff(lqv324_d_f[['userId', 'total_spent']], lqv324_d_f[donate_columns].to_numpy(), 'lqv324', vote_columns)
# lqv324_d_pair.to_csv(os.path.join(OUTPUT_PATH, 'lqv324_d_pair.csv'))

In [ ]:
unlimited_d_pair = survey_diff(unlimited_d_f[['userId', 'total_spent']], unlimited_d_f[donate_columns].to_numpy(), 'unlimited', vote_columns)
# unlimited_d_pair.to_csv(os.path.join(OUTPUT_PATH, 'unlimited_d_pair.csv'))
unlimited_d_pair_credit = survey_diff(unlimited_d_f[['userId', 'total_spent']], unlimited_d_f[donate_columns].to_numpy(), 'unlimited_c', vote_columns)

In [ ]:
all_d_pair = pd.concat([qv36_d_pair, qv108_d_pair, qv324_d_pair, qv36_d_pair_credit, qv108_d_pair_credit, qv324_d_pair_credit, likert_d_pair, lqv36_d_pair, lqv108_d_pair, lqv324_d_pair, unlimited_d_pair, unlimited_d_pair_credit], axis=0).reset_index()[['userId', 'condition', 'topic_1', 'topic_1_val', 'topic_2', 'topic_2_val', 'vote_diff', 'total_spent']]
all_d_pair['vote_diff_norm'] = all_d_pair.apply(normalize_diff_per_row, axis=1, args=('vote_diff', 'total_spent',))

all_d_pair = all_d_pair.rename(columns={"vote_diff": "donation_diff", "vote_diff_norm": "donation_diff_norm"})

print(all_d_pair.head())
print(all_d_pair['donation_diff_norm'].min(), all_d_pair['donation_diff_norm'].max())
print(all_d_pair.shape)
all_d_pair.to_csv(os.path.join(OUTPUT_PATH, 'all_d_pair.csv'))

In [ ]:
frequency_df = all_d_pair.groupby(['userId', 'condition', 'topic_1', 'topic_2']).size().reset_index(name='Frequency')
duplicated_bool = frequency_df['Frequency'] > 1
rows_duplicated = frequency_df[duplicated_bool]
print(rows_duplicated.shape)

In [ ]:
all_d_pair_unique = all_d_pair.drop_duplicates(subset=['userId', 'condition', 'topic_1', 'topic_2'])
print(all_d_pair_unique.shape)

## Join donation and survey dataframes

In [ ]:
all_pair_comp = all_pair_unique.merge(all_d_pair_unique, how='left', on=['userId', 'condition', 'topic_1', 'topic_2'], suffixes=("_vote", "_donation"))
print(all_pair_comp.shape)
all_pair_comp.head()

In [ ]:
frequency_df = all_pair_comp.groupby(['userId', 'condition', 'topic_1', 'topic_2']).size().reset_index(name='Frequency')
rows_duplicated = frequency_df[frequency_df['Frequency'] > 1]
print(rows_duplicated.shape)

In [ ]:
print(all_pair_comp['vote_diff_norm'].min(), all_pair_comp['vote_diff_norm'].max())
print(all_pair_comp['donation_diff_norm'].min(), all_pair_comp['donation_diff_norm'].max())
# print(all_pair_comp['vote_diff_ordinal'].value_counts())

In [ ]:
sns.histplot(all_pair_comp['vote_diff_norm'])
plt.ylim(0, 3000)
plt.show()

In [ ]:
sns.histplot(all_pair_comp['donation_diff_norm'])
plt.ylim(0, 2000)
plt.show()

In [ ]:
print(all_pair_comp[all_pair_comp['donation_diff_norm'] == 0].shape)

In [ ]:
print(all_pair_comp[(all_pair_comp['donation_diff_norm'] == 0) & (all_pair_comp['topic_1_val_donation'] == 0) & (all_pair_comp['topic_2_val_donation'] == 0)].shape)

#### Separate Ordinal and Continuous Conditions

In [61]:
# Ordinal conditions
all_pair_comp_ordinal = all_pair_comp[all_pair_comp['condition'].isin(['qv36', 'qv108', 'qv324', 'likert'])]

def add_ordinal_level(row, ordinal_lvl_dict):
  condition = row['condition']
  ordinal_lvl = ordinal_lvl_dict[condition]
  return ordinal_lvl

ordinal_lvl_dict = {
    'qv36': 17,
    'qv108': 29,
    'qv324':51,
    'likert': 9
}

all_pair_comp_ordinal['ordinal_lvl'] = all_pair_comp_ordinal.apply(add_ordinal_level, axis=1, args=(ordinal_lvl_dict,))

all_pair_comp_ordinal.head()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,userId,condition,order,topic_1,topic_1_val_vote,topic_2,topic_2_val_vote,vote_diff,vote_diff_norm,vote_diff_ordinal,topic_1_val_donation,topic_2_val_donation,donation_diff,total_spent,donation_diff_norm,ordinal_lvl
0,5e62c4156ffda4487364f751,qv36,first,pets,2,art,0,2.0,0.250,10,0,0,0,5,0.000000,17
1,5e640f2eae7e33fbc93cdd09,qv36,decrease,pets,1,art,1,0.0,0.000,8,0,0,0,10,0.000000,17
2,5e64113fae7e33fbc93cdd10,qv36,decrease,pets,1,art,1,0.0,0.000,8,10,0,10,35,0.285714,17
3,5e6488ec23530f5bd5b24194,qv36,decrease,pets,1,art,2,-1.0,-0.125,7,0,0,0,4,0.000000,17
4,5e64adba54fc700d6d481f58,qv36,decrease,pets,2,art,0,2.0,0.250,10,5,0,5,10,0.500000,17


In [62]:
# Further breakdown ordinal conditions, one condition per dataframe
qv36_pair_ordinal = all_pair_comp_ordinal[all_pair_comp_ordinal['condition'].isin(['qv36'])]
qv108_pair_ordinal = all_pair_comp_ordinal[all_pair_comp_ordinal['condition'].isin(['qv108'])]
qv324_pair_ordinal = all_pair_comp_ordinal[all_pair_comp_ordinal['condition'].isin(['qv324'])]
likert_pair_ordinal = all_pair_comp_ordinal[all_pair_comp_ordinal['condition'].isin(['likert'])]

In [63]:
# Continuous conditions
all_pair_comp_continuous = all_pair_comp[all_pair_comp['condition'].isin(['qv36_c', 'qv108_c', 'qv324_c', 'lqv36', 'lqv108', 'lqv324', 'unlimited', 'unlimited_c'])]

def add_condition_id(row, condition_id_dict):
  condition = row['condition']
  condition_id = condition_id_dict[condition]
  return condition_id

# Index(['likert', 'lqv108', 'lqv324', 'lqv36', 'qv108', 'qv108_c', 'qv324',
#   'qv324_c', 'qv36', 'qv36_c', 'unlimited', 'unlimited_c'],

condition_id_dict = {
    'qv36_c': 9,
    'qv108_c': 5,
    'qv324_c': 7,
    'lqv36': 3,
    'lqv108': 1,
    'lqv324': 2,
    'unlimited': 10,
    'unlimited_c': 11
}

all_pair_comp_continuous['condition_id'] = all_pair_comp_continuous.apply(add_condition_id, axis=1, args=(condition_id_dict,))

all_pair_comp_continuous.head()

all_pair_comp_continuous.to_csv(os.path.join(OUTPUT_PATH, 'all_pair_comp_continuous.csv'))

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Model

### Model as a mix of ordinal and continous variables

In [64]:
# As continuous (all credits, linear QV votes, unlimited votes) and ordinal (for QV votes and likert)
def pair_diff_strength_continuous_ordinal(vote_diff_norm, vote_diff_qv36, vote_diff_qv108, vote_diff_qv324, vote_diff_likert, condition, topic_1, topic_2, order, donation_diff=None):
    # vote_diff_ordinal_levels is a list of integers, each represents the ordinal level of a condition

    NC = 12
    NT = 9
    NO = 3

    ########################### Priors ##################################

    # ##### Intercepts #####
    # alpha = npr.sample("alpha", dist.Normal(0.0, 0.1))

    ##### Conditions varying intercepts #####

    # # hyper priors
    # b_cond_bar = npr.sample("b_cond_bar", dist.Normal(0.0, 0.1))
    # sigma_cond = npr.sample("sigma_cond", dist.Uniform(0, 1))

    # # adaptive prior (non-centered)
    # b_cond_z = npr.sample("b_cond_z", dist.Normal(0.0, 1.0).expand([NC]))
    # b_cond = npr.deterministic("b_cond", jnp.add(b_cond_bar, jnp.multiply(b_cond_z, sigma_cond)))
    b_cond = npr.sample("b_cond", dist.Normal(0.0, 0.2).expand([NC]))

    ##### Conditions x Vote difference varying slopes #####
    # Index(['likert', 'lqv108', 'lqv324', 'lqv36', 'qv108', 'qv108_c', 'qv324',
    #   'qv324_c', 'qv36', 'qv36_c', 'unlimited', 'unlimited_c'],

    # hyper priors
    b_vote_bar = npr.sample("b_vote_bar", dist.Normal(0.0, 1.0))
    sigma_vote = npr.sample("sigma_vote", dist.Uniform(0, 1))

    b_vote_z = npr.sample("b_vote_z", dist.Normal(0.0, 1.0).expand([NC]))
    b_vote = npr.deterministic("b_vote", jnp.add(b_vote_bar, jnp.multiply(b_vote_z, sigma_vote)))

    # b_vote = npr.sample("b_vote", dist.Normal(0.0, 0.5).expand([NC]))

    # Ordinal conditions

    a_cf_qv36 = npr.sample("a_cf_qv36", dist.Dirichlet(jnp.repeat(2, 17 - 1)))
    a_cf_k_qv36 = jnp.pad(a_cf_qv36, (1, 0))
    a_cf_k_cumulative_qv36 = jnp.sum(jnp.where(jnp.arange(17) <= vote_diff_qv36[..., None], a_cf_k_qv36, 0), -1)
    # N_qv36 = len(vote_diff_qv36)
    # b_vote_arr_qv36 = jnp.repeat(b_vote[8], N_qv36)

    a_cf_qv108 = npr.sample("a_cf_qv108", dist.Dirichlet(jnp.repeat(2, 29 - 1)))
    a_cf_k_qv108 = jnp.pad(a_cf_qv108, (1, 0))
    a_cf_k_cumulative_qv108 = jnp.sum(jnp.where(jnp.arange(29) <= vote_diff_qv108[..., None], a_cf_k_qv108, 0), -1)
    # N_qv108 = len(vote_diff_qv108)
    # b_vote_arr_qv108 = jnp.repeat(b_vote[4], N_qv108)

    a_cf_qv324 = npr.sample("a_cf_qv324", dist.Dirichlet(jnp.repeat(2, 51 - 1)))
    a_cf_k_qv324 = jnp.pad(a_cf_qv324, (1, 0))
    a_cf_k_cumulative_qv324 = jnp.sum(jnp.where(jnp.arange(51) <= vote_diff_qv324[..., None], a_cf_k_qv324, 0), -1)
    # N_qv324 = len(vote_diff_qv324)
    # b_vote_arr_qv324 = jnp.repeat(b_vote[6], N_qv324)

    a_cf_likert = npr.sample("a_cf_likert", dist.Dirichlet(jnp.repeat(2, 9 - 1)))
    a_cf_k_likert = jnp.pad(a_cf_likert, (1, 0))
    a_cf_k_cumulative_likert = jnp.sum(jnp.where(jnp.arange(9) <= vote_diff_likert[..., None], a_cf_k_likert, 0), -1)
    # N_likert = len(vote_diff_likert)
    # b_vote_arr_likert = jnp.repeat(b_vote[0], N_likert)

    # # Continuous conditions
    # b_vote_continuous = jnp.take(b_vote, continuous_condition_id)

    # Concatenate all conditions
    # b_vote_arr = jnp.concatenate((b_vote_arr_qv36, b_vote_arr_qv108, b_vote_arr_qv324, b_vote_arr_likert, b_vote_continuous))
    b_vote_arr = b_vote[condition]
    vote_diff_arr = jnp.concatenate((a_cf_k_cumulative_qv36, a_cf_k_cumulative_qv108, a_cf_k_cumulative_qv324, a_cf_k_cumulative_likert, vote_diff_norm))
    # print(b_vote_arr.shape)
    # print(vote_diff_arr.shape)
    # array_min = jnp.min(vote_diff_arr)
    # array_max = jnp.max(vote_diff_arr)
    # print(f"Min: {array_min}; Max: {array_max}")


    ##### Order varying intercepts #####

    # hyper priors
    b_order_bar = npr.sample("b_order_bar", dist.Normal(0.0, 0.1))
    sigma_order = npr.sample("sigma_order", dist.Uniform(0, 1))

    # adaptive prior (non-centered)
    b_order_z = npr.sample("b_order_z", dist.Normal(0.0, 1.0).expand([NO]))
    b_order = npr.deterministic("b_order", jnp.add(b_order_bar, jnp.multiply(b_order_z, sigma_order)))

    ##### Topic varying intercepts #####

    # hyper priors
    b_topic_bar = npr.sample("b_topic_bar", dist.Normal(0.0, 0.1))
    sigma_topic = npr.sample("sigma_topic", dist.Uniform(0, 1))

    # adaptive prior (non-centered)
    b_topic_z = npr.sample("b_topic_z", dist.Normal(0.0, 1.0).expand([NT]))
    b_topic = npr.deterministic("b_topic", jnp.add(b_topic_z, jnp.multiply(b_topic_bar, sigma_topic)))


    ##### Sigma intercept #####
    b_sigma = npr.sample("b_sigma", dist.Exponential(1).expand([NC]))


    ################ mean and std of observed data #############

    mu = npr.deterministic('mu', b_cond[condition] + b_vote_arr * vote_diff_arr + b_order[order] +
                            b_topic[topic_1] + b_topic[topic_2])

    sigma = npr.deterministic('sigma', b_sigma[condition])

    npr.sample('donation_diff', dist.Normal(mu, sigma), obs=donation_diff)

In [65]:
condition = pd.concat([qv36_pair_ordinal['condition'], qv108_pair_ordinal['condition'], qv324_pair_ordinal['condition'], likert_pair_ordinal['condition'], all_pair_comp_continuous['condition']], ignore_index=True)
order = pd.concat([qv36_pair_ordinal['order'], qv108_pair_ordinal['order'], qv324_pair_ordinal['order'], likert_pair_ordinal['order'], all_pair_comp_continuous['order']], ignore_index=True)
topic_1 = pd.concat([qv36_pair_ordinal['topic_1'], qv108_pair_ordinal['topic_1'], qv324_pair_ordinal['topic_1'], likert_pair_ordinal['topic_1'], all_pair_comp_continuous['topic_1']], ignore_index=True)
topic_2 = pd.concat([qv36_pair_ordinal['topic_2'], qv108_pair_ordinal['topic_2'], qv324_pair_ordinal['topic_2'], likert_pair_ordinal['topic_2'], all_pair_comp_continuous['topic_2']], ignore_index=True)

condition_idx, condition_label = pd.factorize(condition, sort=True)
order_idx, order_label = pd.factorize(order, sort=True)
topic_1_idx, topic_1_label = pd.factorize(topic_1, sort=True)
topic_2_idx, topic_2_label = pd.factorize(topic_2, sort=True)

predict_data_dict = {
    'condition': jnp.asarray(condition_idx),
    'topic_1': jnp.asarray(topic_1_idx),
    'topic_2': jnp.asarray(topic_2_idx),
    "order": jnp.asarray(order_idx),
    "vote_diff_norm": jnp.asarray(all_pair_comp_continuous['vote_diff_norm']),
    "vote_diff_qv36": jnp.asarray(qv36_pair_ordinal['vote_diff_ordinal']),
    "vote_diff_qv108": jnp.asarray(qv108_pair_ordinal['vote_diff_ordinal']),
    "vote_diff_qv324": jnp.asarray(qv324_pair_ordinal['vote_diff_ordinal']),
    "vote_diff_likert": jnp.asarray(likert_pair_ordinal['vote_diff_ordinal']),
}

donation_diff = pd.concat([qv36_pair_ordinal['donation_diff_norm'], qv108_pair_ordinal['donation_diff_norm'], qv324_pair_ordinal['donation_diff_norm'], likert_pair_ordinal['donation_diff_norm'], all_pair_comp_continuous['donation_diff_norm']], ignore_index=True)

data_dict_DV = {
    'donation_diff': jnp.asarray(donation_diff)
}

fitting_data_dict = {**predict_data_dict, **data_dict_DV}

print(condition_label)
print(order_label)
print(topic_1_label)

Index(['likert', 'lqv108', 'lqv324', 'lqv36', 'qv108', 'qv108_c', 'qv324',
       'qv324_c', 'qv36', 'qv36_c', 'unlimited', 'unlimited_c'],
      dtype='object')
Index(['decrease', 'first', 'increase'], dtype='object')
Index(['art', 'education', 'environment', 'faith', 'health', 'human',
       'international', 'pets'],
      dtype='object')


In [66]:
dims = {
    "b_cond": ["conditions"],
    "b_vote": ["conditions"],
    "b_topic": ["topics"],
    "b_order": ["orders"],
    "b_sigma": ["conditions"]
}

conditions = condition_label.tolist()
orders = order_label.tolist()
topics = ['art', 'education', 'environment', 'faith', 'health', 'human', 'international', 'pets', 'veteran']

coords = {
    "conditions": conditions,
    "topics": topics,
    "orders": orders
}


In [67]:
pair_diff_strength_model = pair_diff_strength_continuous_ordinal
kernel = NUTS(pair_diff_strength_model, target_accept_prob=0.95)
sample_kwargs = dict(
    sampler=kernel,
    num_warmup=500,
    num_samples=2000,
    num_chains=4,
    chain_method="parallel"
)



In [68]:
rng_key = random.PRNGKey(0)
all_mcmc = MCMC(**sample_kwargs)
all_mcmc.run(rng_key, **fitting_data_dict)

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

In [69]:
prior = Predictive(all_mcmc.sampler.model, num_samples=1000)(random.PRNGKey(1), **predict_data_dict)

In [70]:
posterior_samples = all_mcmc.get_samples()

In [ ]:
posterior_predictive = Predictive(all_mcmc.sampler.model, posterior_samples)(random.PRNGKey(2), **predict_data_dict)

In [ ]:
MODEL_PATH='.'
MODEL_PATH + '/numpyro_intensity_ord_cont_model.trace'

In [ ]:
model_data = az.from_numpyro(
    posterior=all_mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive,
    coords=coords,
    dims=dims,
)
# Save trace
az.to_netcdf(data=model_data, filename=MODEL_PATH + '/numpyro_intensity_ord_cont_model.trace')

##### Result

In [2]:
model_data = az.from_netcdf('./numpyro_intensity_ord_cont_model.trace')

In [6]:
def plot_posterior_predictive_cumulative(data, xlabel):
  axs = az.plot_ppc(data=data,
              group='posterior',
              kind='cumulative',
              random_seed=5,
              colors=[neutral_gray, base_color, accent_color])

  z = axs
  axs.set_title('Posterior Predictive', size=12)
  z.set_xlabel(xlabel, fontsize=16)  # no individual labels
  z.set_ylabel("Density", fontsize=16)
  z.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
  z.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
  z.spines['right'].set_visible(False)
  z.spines['top'].set_visible(False)
  z.spines['left'].set_visible(False)
  z.spines['bottom'].set_visible(False)
  z.spines['bottom'].set_position(('outward', 10))
  z.xaxis.set_minor_locator(AutoMinorLocator(5))
  z.set_xlim(left=-2, right=2)
  z.tick_params(which='both', width=2)
  z.tick_params(which='major', length=7)
  z.tick_params(which='minor', length=4, color=neutral_gray)

  # plt.subplots_adjust(hspace=0.75, wspace=0.1)
  # filename = 'plots/' +Title + '.pdf'
  # plt.savefig(filename, transparent=False)
  plt.show()
  # rc('text', usetex=False)


plot_posterior_predictive_cumulative(model_data, 'Normalized Donation Difference')

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe8eeac6020>>
Traceback (most recent call last):
  File "/anaconda/envs/jupyter_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [3]:
az.summary(data=model_data,
          var_names=[
              # 'b_cond_bar', 'sigma_cond',
              'b_cond',
              # 'b_vote_bar', 'sigma_vote',
              'b_vote',
              'b_topic_bar', 'sigma_topic', 'b_topic',
              'b_order_bar', 'sigma_order', 'b_order',
              'b_sigma'
          ],
           round_to=2
          )

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
b_cond[likert],-0.16,0.07,-0.31,-0.03,0.00,0.00,2675.59,4477.34,1.0
b_cond[lqv108],0.15,0.06,0.04,0.26,0.00,0.00,1651.37,2993.38,1.0
b_cond[lqv324],0.14,0.06,0.03,0.25,0.00,0.00,1646.79,2958.34,1.0
b_cond[lqv36],0.14,0.06,0.03,0.25,0.00,0.00,1648.79,2981.89,1.0
b_cond[qv108],-0.32,0.08,-0.48,-0.17,0.00,0.00,3087.74,5022.25,1.0
b_cond[qv108_c],0.14,0.06,0.03,0.25,0.00,0.00,1635.92,2957.29,1.0
b_cond[qv324],-0.36,0.07,-0.49,-0.22,0.00,0.00,2700.46,4925.70,1.0
b_cond[qv324_c],0.12,0.06,0.01,0.23,0.00,0.00,1654.00,2996.05,1.0
b_cond[qv36],-0.29,0.09,-0.45,-0.12,0.00,0.00,3524.17,4960.01,1.0
b_cond[qv36_c],0.15,0.06,0.04,0.26,0.00,0.00,1640.96,2964.15,1.0
